In [ ]:
import sys, os

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import pickle

In [ ]:
sys.path.append(os.path.abspath('../../'))
from utils import utils
from utils.utils import evaluate_experiment
sys.path.append(os.path.abspath('../'))
from timeseries_utils import *

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

In [ ]:
path_to_data='../../../'

In [ ]:
data=np.load(os.path.abspath(path_to_data+'data-002.npy'),allow_pickle=True)
Y=np.load(path_to_data+'Y.npy',allow_pickle=True)
labels =pd.read_csv(path_to_data+'labels.csv')

In [ ]:
train_fold=8
val_fold=9
test_fold=10

# 10th fold for testing (9th for now)
X_test = data[labels.strat_fold == test_fold]
y_test = Y[labels.strat_fold == test_fold]
# 9th fold for validation (8th for now)
X_val = data[labels.strat_fold == val_fold]
y_val = Y[labels.strat_fold == val_fold]
# rest for training
X_train = data[labels.strat_fold <= train_fold]
y_train = Y[labels.strat_fold <= train_fold]

In [ ]:
X_train.shape ,X_val.shape

In [ ]:
# Preprocess signal data
X_train, X_val, X_test = utils.preprocess_signals(X_train, X_val, X_test,'/content/')
n_classes = y_train.shape[1]
X_train = np.reshape(X_train,[X_train.shape[0],X_train.shape[2],X_train.shape[1]])
X_val = np.reshape(X_val,[X_val.shape[0],X_val.shape[2],X_val.shape[1]])
X_test = np.reshape(X_test,[X_test.shape[0],X_test.shape[2],X_test.shape[1]])

In [ ]:
X_train.shape, X_val.shape

In [ ]:
y_train.shape, y_val.shape

In [ ]:
X_train = X_train.reshape((X_train.shape[0]*X_train.shape[1],X_train.shape[2]))

In [ ]:
X_val = X_val.reshape((X_val.shape[0]*X_val.shape[1],X_val.shape[2]))

In [ ]:
X_train.shape, X_val.shape

In [ ]:
y1=[]
for y in y_train:
  for i in range(12):
    res = int("".join(str(k) for k in y), 2)
    y1.append(res)
y1=np.asarray(y1)
y1.shape

In [ ]:
y2=[]
for y in y_val:
  for i in range(12):
    res = int("".join(str(k) for k in y), 2)
    y2.append(res)
y2=np.asarray(y2)
y2.shape

In [ ]:
tpot = TPOTClassifier(generations=100,
                      population_size=100,
                      verbosity=2,
                      random_state=1223,
                      warm_start=True,
                      memory='./save/',
                      use_dask=True,
                      n_jobs=3,
                      periodic_checkpoint_folder='./save/',
                      max_time_mins=None,
                      max_eval_time_mins=5, 
                      early_stop=20)

In [ ]:
x_aux = X_train[X_train.shape[0]//3:]
y_aux = y1[X_train.shape[0]//3:]

In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(x_aux, y_aux, random_state=1223)

In [ ]:
tpot.fit(X, y)